In [1]:
from bs4 import BeautifulSoup
import re
import requests
import fake_useragent
import json
import csv
import random
import time
import os
import pandas as pd

ua = fake_useragent.UserAgent( )

headers = {
    "Accept": "*/*",
    "User-Agent": ua.ie,
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
}

In [20]:
#СОХРАНИТЬ ПАРАМЕТРЫ ДАТЧИКОВ С КАЖДОЙ СТРАНИЦЫ
# with open(f'data/Индуктивные датчики TEKO и LANBAO.csv', encoding = 'utf-8') as f:
#     df_lanbao_lanbao = pd.read_csv(f)

with open(f'data/Индуктивные датчики_LANBAO.csv', encoding = 'utf-8') as f:
    df_lanbao_lanbao = pd.read_csv(f)
    
df_lanbao_lanbao.loc [df_lanbao_lanbao.sensor_symbol.str.contains('^[LF]'), 'manufacturer'] = 'LANBAO'
df_lanbao_lanbao = df_lanbao_lanbao.loc[df_lanbao_lanbao['manufacturer'] == 'LANBAO']                                         #Только датчики LANBAO
df_lanbao_lanbao.drop_duplicates(subset=['sensor_symbol'], keep='first', inplace=True)

# df_lanbao_lanbao = df_lanbao_lanbao.assign(param = None)


for index, row in df_lanbao_lanbao.iterrows():
    if not df_lanbao_lanbao['param'].isnull()[index]:
        continue
    parameters_dict = {}
    param_req = requests.get(row['sensor_url'], headers=headers)
    param_soup = BeautifulSoup(param_req.content, 'lxml')
    
    parameters = param_soup.find('table', id="atributipodbr").find_all('tr', class_='niddle')
    for item in parameters:
        parameter = item.find('td', class_='levocol').get_text().strip()
        value = item.find('td', class_='pravokol').get_text().strip()
        parameters_dict[parameter] = value
    

    df_lanbao_lanbao.loc [df_lanbao_lanbao.index == index, 'param'] = json.dumps(parameters_dict, ensure_ascii=False)
    print(index, parameters_dict)

    
    time.sleep(random.randrange(3,8))

df_lanbao_lanbao.to_csv('data/Индуктивные датчики_LANBAO.csv', index=False)

608 {'Тип монтажа': 'Встраиваемый', 'Рабочее расстояние срабатывания': '25 мм', 'Способ подключения': 'Разъемно-штекерное', 'Материал корпуса': 'PBT', 'Тип корпуса': 'Прямоугольный', 'Номинальное напряжение питания, Uном.': '10...30 В DC', 'Тип контакта / Структура выхода': 'NPN Переключающий', 'Присоединение / Подключение': 'Разъём M12 4pin', 'Степень защиты по ГОСТ 14254-96': 'IP67', 'Диапазон рабочих температур': '-25°С...+70°С', 'Принцип действия': 'Увеличенное расстояние срабатывания'}
609 {'Тип монтажа': 'Встраиваемый', 'Рабочее расстояние срабатывания': '25 мм', 'Способ подключения': 'Разъемно-штекерное', 'Материал корпуса': 'PBT', 'Тип корпуса': 'Прямоугольный', 'Номинальное напряжение питания, Uном.': '10...30 В DC', 'Тип контакта / Структура выхода': 'NPN Замыкающий', 'Присоединение / Подключение': 'Разъём M12 4pin', 'Степень защиты по ГОСТ 14254-96': 'IP67', 'Диапазон рабочих температур': '-25°С...+70°С', 'Принцип действия': 'Увеличенное расстояние срабатывания'}
610 {'Тип м

In [125]:
def getimage(url, index, df_lanbao_lanbao):
    if df_lanbao_lanbao['sensor_imgurl'].isnull()[index]:
        return '/static/main/img/ind_LANBAO/Lanbao.png'
    file_name = url.split('/')[-1]
    file_adr = '../main/static/main/img/ind_LANBAO/' + file_name
    if not os.path.exists(file_adr):
        time.sleep(random.randrange(4,10))
        img_data = requests.get(url).content
        with open(file_adr, 'wb') as handler:
            handler.write(img_data)
    return '/static/main/img/ind_LANBAO/' + file_name

def getdistance(parameters_dict):
    dist = parameters_dict.get('Рабочее расстояние срабатывания')
    match = re.search(r"^(\d+,?\d?).*", dist)
    if match:
        return match.group(1)
    return dist

def getsize(parameters_dict, name):
    pattern = re.compile(r'LE80XZS[NF][75]0D[PN]RY')
    if pattern.findall(name):
        return '40x80x104'
    pattern = re.compile(r'LE40XZSF25D[NP][CRO]Y')
    if pattern.findall(name):
        return '40x40x140'
    pattern = re.compile(r'LE40XZSN40D[NP][CRO]Y-')
    if pattern.findall(name):
        return '40x40x129'
    pattern = re.compile(r'LE40SZS[NF](40|25)D[NP][CRO]Y')
    if pattern.findall(name):
        return '40x40x66'
    size = parameters_dict.get('Размер прямоугольного корпуса, ДлxВxШ')
    if size: 
        return size
    size = parameters_dict.get('Диаметр резьбового корпуса')
    if size: 
        match = re.search(r"^[MМ](\d+).*", size)
        if match:
            return '∅' + match.group(1).replace('.', ',')
    size = parameters_dict.get('Диаметр гладкого корпуса')
    if size and parameters_dict.get('Тип корпуса') == 'Цилиндрический':
        match = re.search(r"^Ø(\d+\.?\d?).*", size)
        if match:
            return '∅' + match.group(1).replace('.', ',')
    if size and parameters_dict.get('Тип корпуса') == 'Цилиндрический резьбовой':
        match = re.search(r"^[MМ](\d+).*", size)
        if match:
            return '∅' + match.group(1).replace('.', ',')
    return None
    

def getoutputstructure(parameters_dict):
    structure = parameters_dict.get('Тип контакта / Структура выхода')
    match = re.search(r"^(P?NPN?).*", structure)
    if match:
        return match.group(1)
    return structure

def getoutputtype(parameters_dict):
    structure = parameters_dict.get('Тип контакта / Структура выхода')
    if 'Размыкающий' in structure:
        return 'NC'
    if 'Замыкающий' in structure:
        return 'NO'
    if 'Переключающий' in structure:
        return 'NO и NC'
    return None

def gethousing(parameters_dict, name):
    size = parameters_dict.get('Диаметр гладкого корпуса')
    pattern = re.compile(r'(4|\.5)[XQ]')
    if size and pattern.findall(name):
        return 'Цилиндрический гладкий'
    housing = str(parameters_dict.get('Тип корпуса'))
    if housing == 'Цилиндрический резьбовой':
        return 'Цилиндрический с резьбой'
    return housing


def getmaterial(parameters_dict):
    material = str(parameters_dict.get('Материал корпуса'))
    material = material.replace('PBT', 'Пластик').replace('PC', 'Пластик')
    return material

def getspec(parameters_dict):
    if (parameters_dict.get('Принцип действия')) == 'Высокое давление':
        return 'В среде высокого давления'
    return 'Высокая частота переключения' if 'FY' in row['sensor_symbol'] else 'Общее применение'


In [126]:
with open(f'data/Индуктивные датчики_LANBAO.csv', encoding = 'utf-8') as f:
    df_lanbao = pd.read_csv(f)
    
df_lanbao.drop_duplicates(subset=['sensor_symbol'], keep='first', inplace=True)

df_lanbao = df_lanbao.assign(manufacturer = 'LANBAO' ,
                       flush = None,
                       spec = None,
                       housing = None,
                       connection = None,
                       size = None,
                       material = None,
                       IP = None,
                       output_structure = None,
                       voltage = None,
                       output_type = None,
                       adjust = 'Нет',
                       distance = None,
                       indicator = 'Есть',
                       temperature = None)

df_lanbao.loc [df_lanbao.sensor_in_stock.str.contains('Пожалуйста'), 'sensor_in_stock'] = 'По запросу'


for index, row in df_lanbao.iterrows():
    parameters_dict = {}
    parameters_dict = json.loads(row ['param'])

    df_lanbao.loc[df_lanbao.index == index, 'sensor_type'] = 'Индуктивный датчик'
    df_lanbao.loc[df_lanbao.index == index, 'flush'] = str(parameters_dict.get('Тип монтажа'))
    df_lanbao.loc[df_lanbao.index == index, 'spec'] = getspec(parameters_dict)
    
    df_lanbao.loc[df_lanbao.index == index, 'housing'] = gethousing(parameters_dict, row['sensor_symbol'])
    df_lanbao.loc[df_lanbao.index == index, 'size'] = getsize(parameters_dict, row['sensor_symbol'])
    df_lanbao.loc[df_lanbao.index == index, 'connection'] =str(parameters_dict.get('Способ подключения')).replace('Разъемно-штекерное', 'Разъём')
    df_lanbao.loc[df_lanbao.index == index, 'material'] = getmaterial(parameters_dict)
    df_lanbao.loc[df_lanbao.index == index, 'IP'] = str(parameters_dict.get('Степень защиты по ГОСТ 14254-96')).replace('IP', 'IP ')
    df_lanbao.loc[df_lanbao.index == index, 'output_structure'] = getoutputstructure(parameters_dict)
    df_lanbao.loc[df_lanbao.index == index, 'output_type'] = getoutputtype(parameters_dict)
    df_lanbao.loc[df_lanbao.index == index, 'distance'] = getdistance(parameters_dict)
    df_lanbao.loc[df_lanbao.index == index, 'temperature'] = str(parameters_dict.get('Диапазон рабочих температур')).replace('°С', '')
    df_lanbao.loc[df_lanbao.index == index, 'voltage'] = str(parameters_dict.get('Номинальное напряжение питания, Uном.')).replace('10...30 В DC', '10-30 [DC]').replace('20…30 В DC', '20-30 [DC]')

    df_lanbao.loc[index, 'sensor_imgurl'] = getimage(row['sensor_imgurl'], index, df_lanbao)

    print(index, row['sensor_symbol'])

df_lanbao['sensor_price'] = df_lanbao['sensor_price'].str.replace(' ','')
df_lanbao['sensor_price'] = df_lanbao.sensor_price.str.extract(r'([0-9]+)')
df_lanbao['sensor_price'] = df_lanbao['sensor_price'].str.replace(',','.')

df_lanbao['distance'] = df_lanbao['distance'].str.replace(',','.').astype(float)

df_lanbao.drop_duplicates(subset=['sensor_symbol'], keep='first', inplace=True)

df_lanbao.drop(columns='param', inplace=True)

df_lanbao = df_lanbao.reindex(columns=['category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_imgurl',
                         'sensor_price', 'sensor_in_stock', 'manufacturer', 'flush', 'spec', 'housing',
                         'connection', 'size', 'material', 'IP','output_structure','voltage',
                         'output_type','adjust','distance','indicator','temperature'])

new_columns = {'category_name': 'category',
               'sensor_type': 'type',
               'sensor_symbol': 'name',
               'sensor_url': 'url',
               'sensor_imgurl': 'url_img',
               'sensor_price': 'price',
               'sensor_in_stock': 'stock',
               }

df_lanbao = df_lanbao.rename(columns=new_columns)

df_lanbao.to_csv('data/Индуктивные датчики LANBAO dataframe.csv', index=False)

0 LE17SF05DNO
1 LE80XZSN70DPRY-E2
2 LE80XZSN70DPRY-D
3 LE80XZSN70DNRY-E2
4 LE80XZSN70DNRY-D
5 LE80XZSN50DPRW1-E2
6 LE80XZSN50DPRW1-D
7 LE80XZSN50DPOW1-E2
8 LE80XZSN50DPOW1-D
9 LE80XZSN50DPCW1-E2
10 LE80XZSN50DPCW1-D
11 LE80XZSN50DNRW1-E2
12 LE80XZSN50DNRW1-D
13 LE80XZSN50DNOW1-E2
14 LE80XZSN50DNOW1-D
15 LE80XZSN50DNCW1-E2
16 LE80XZSN50DNCW1-D
17 LE80XZSF50DPRY-E2
18 LE80XZSF50DPRY-D
19 LE80XZSF50DNRY-E2
20 LE80XZSF50DNRY-D
21 LE80XZSF40DPRW1-E2
22 LE80XZSF40DPRW1-D
23 LE80XZSF40DPOW1-E2
24 LE80XZSF40DPOW1-D
25 LE80XZSF40DPCW1-E2
26 LE80XZSF40DPCW1-D
27 LE80XZSF40DNRW1-E2
28 LE80XZSF40DNRW1-D
29 LE80XZSF40DNOW1-D
30 LE80XZSF40DNCW1-E2
31 LE80XZSF40DNCW1-D
32 LE40XZSF25DPRY-D
33 LE40XZSF25DPOY-D
34 LE40XZSF25DPCY-D
35 LE40XZSF25DNRY-D
36 LE40XZSF25DNOY-D
37 LE40XZSF25DNCY-D
38 LR6.5QBN04DPOY-E1
39 LR6.5QBN04DPOY
40 LR6.5QBN04DPCY-E1
41 LR6.5QBN04DPCY
42 LR6.5QBN04DNOY-E1
43 LR6.5QBN04DNOY
44 LR6.5QBN04DNCY-E1
45 LR6.5QBN04DNCY
46 LR6.5QBN02DPO-E1
47 LR6.5QBN02DPO
48 LR6.5QBN02DPC-E1
49 L

In [121]:
df_lanbao['temperature'].unique()

# 'category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_imgurl',
# 'sensor_price', 'sensor_in_stock', 'manufacturer', 'flush', 'spec', 'housing',
# 'connection', 'size', 'material', 'IP','output_structure','voltage',
# 'output_type','adjust','distance','indicator','temperature'

array(['-25...+70', '-40...+70', '-25...+120', '-25...+80'], dtype=object)